In [ ]:
# Run this cell to install DiffeRT and its dependencies, e.g., on Google Colab

try:
    import differt  # noqa: F401
except ImportError:
    import sys  # noqa: F401

    !{sys.executable} -m pip install differt[all]

(smoothing)=
# Smoothing Discontinuities for Fully Differentiable Ray Tracing

In a previous work {cite}`fully-eucap2024`, we introduce a smoothing technique that aims
at smoothing the discontinuities caused by Ray Tracing in the hope to make optimization
through gradient descent easier. This technique was originally implemented in {cite}`differt2d`,
and we now aim to reproduce a similar implementation inside DiffeRT, but with less flexibility to
reduce code complexity.

We will not go into details of how the method actually work, nor the actual implementation details,
but you can always read the source code or the original paper {cite}`fully-eucap2024` for more information.

## Smoothing Function

The smoothing function can be any smooth function {math}`s: (x;\alpha) \in \mathbb{R} \times \mathbb{R}^+ \mapsto s(x;\alpha) \in [0;1]`, with {math}`s \in C^1`, with the following properties on the smoothing factor {math}`\alpha`:

:::{math}
    \lim_{\alpha\rightarrow\infty} s(x;\alpha) = \theta(x),
:::

where

:::{math}
\theta(x) = \begin{cases} 1, &\text{if }x>0,\\ 0, &\text{otherwise.}\end{cases}
:::

In practice, additional constraints are imposed to {math}`s`, but they are not detailed here.
Evaluating {math}`s(0;\alpha)` is considered to be *undefined behavior*, even if it returns actual values (e.g., {math}`\frac{1}{2}`), as it might change in the future.

## Upstream functions using smoothing

Currently, only a limited set of functions allow returning a *smoothed-out* value
instead of a hard boolean decision. The most important function is
{meth}`TriangleScene.compute_paths<differt.scene.TriangleScene.compute_paths>`. The rest
are downstream functions used by the latter, and can be identified with their optional
`smoothing_factor` argument.

## Boolean equivalents

The following boolean comparisons are evaluated with the following equivalent real-value functions:

:::{table} Real-valued equivalents to common boolean operators
:widths: auto
:align: center

| boolean | real-valued |
| --- | --- |
| `x \| y` | {func}`jnp.minimum(x, y)<jax.numpy.minimum>` |
| `x & y` | {func}`jnp.maximum(x, y)<jax.numpy.maximum>` |
| `x > y` | {func}`s(x - y)<differt.utils.smoothing_function>` |
| `x < y` | {func}`s(y - x)<differt.utils.smoothing_function>` |
| `x >= y` | {func}`s(x - y)<differt.utils.smoothing_function>` |
| `x <= y` | {func}`s(y - x)<differt.utils.smoothing_function>` |
:::

Note the lack of stick equality operator, as it is (currently) not required by upstream functions.


## Examples

In this section, we show usage examples of the smoothing techniques.

### Coverage Map

TODO

### Optimizing TX Position

TODO